In [1]:
import os
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"   
os.environ["CUDA_VISIBLE_DEVICES"]="0"

import torch
import numpy as np
import pickle
import cv2
import os

from torchvision.models import resnet152
from torchvision import transforms
from torch import nn

from common import Data, Split, Batches, load_data, encode_y, load_split
from utils import ProgressBar, evaluate
from pack import Pack
from model import PosterOnlyModel, Model
from torch_models import MultiLayerFCReLUClassifier
from itertools import chain
from train import train_epoches

from cls import BRClassifier

%load_ext autoreload
%autoreload 2

In [2]:
genre_list = pickle.load(open("../data/tmdb_genres_list.pkl", 'rb'))

GENRES = load_data("../local/genres.pkl")
train = load_split("../local/train.pkl")
val = load_split("../local/val.pkl")
test = load_split("../local/test.pkl")
POSTERS = load_data("../local/posters.npy")

MODEL = resnet152(pretrained=True)

In [3]:
# Freeze base layers
for child in list(MODEL.children())[:-3]:
    for param in child.parameters():
        param.requires_grad = False

# Conv4, turn off grad in first 2 blocks
for child in list(list(MODEL.children())[-3].children())[:2]:
    for param in child.parameters():
        param.requires_grad = False

In [4]:
CNN = nn.Sequential(*list(MODEL.children())[:-1]).cuda()
classifier = BRClassifier(dims=[1024, 512], num_class=19, encoding_size=2048, cuda=True)

model = PosterOnlyModel(CNN, classifier, POSTERS, GENRES)

In [5]:
adam = torch.optim.Adam(filter(lambda p:p.requires_grad, model.parameters()))

optimizer = adam
scheduler = None

loss = torch.nn.BCEWithLogitsLoss().cuda()

In [ ]:
loss_hist = []
n_epochs = 10
for i in range(0,20):
    epoch_losses = train_epoches(n_epochs=n_epochs, model=model, train=train, loss=loss, val=val,
                  batch_size=32, optimizer=optimizer, scheduler=scheduler)
    loss_hist.append(epoch_losses)
    bn = (i+1)*n_epochs
    torch.save(model.encoder, "saved/poster-res2/encoder_{}_{}.pth".format(bn, str(epoch_losses[1][1][-1])[:4]))
    torch.save(model.classifier, "saved/poster-res2/cls_{}_{}.pth".format(bn, str(epoch_losses[1][1][-1])[:4]))

......................................5.....................................10......................................15.....................................20......................................25.....................................30......................................35.....................................40......................................45.....................................50......................................55.....................................60......................................65.....................................70......................................75.....................................80......................................85.....................................90......................................95.....................................100 F
epoch 0: 0.2053055
Val:	P(mi) 0.09912109375 	R(mi): 0.5858585858585859 	F1(mi): 0.16955523073710588

......................................5.....................................10............................

......................................5.....................................10......................................15.....................................20......................................25.....................................30......................................35.....................................40......................................45.....................................50......................................55.....................................60......................................65.....................................70......................................75.....................................80......................................85.....................................90......................................95.....................................100 F
epoch 8: 0.022308147
Val:	P(mi) 0.41650390625 	R(mi): 0.49607444024425706 	F1(mi): 0.4528201725282017

......................................5.....................................10..........................

In [ ]:
evaluate(test, model, 128)